In [1]:
import tensorflow as tf
import numpy as np
import json as js
import os
import shutil
import random

data = {}
with open('data.json') as file:
    data = js.load(file)
data = [x for x in data if x['d_time'] >= 0]

random.shuffle(data)
def MLP(x):
    init = tf.variance_scaling_initializer()
    layer_1 = tf.layers.dense(x, N_input * 2, activation=tf.nn.relu, kernel_initializer=init)
    layer_2 = tf.layers.dense(layer_1, N_input * 2, activation=tf.nn.relu, kernel_initializer=init)
    out = tf.layers.dense(layer_2, N_output, kernel_initializer=init)
    return out

sum_ac = 0.0
sum_n = 0.0

for user_number in range(208, 404):
    X, T = np.array([[x[y] for y in x if y !='user' and y != 'd_time'] for x in data if x['user'] == 'user{0}'.format(user_number)]), np.array([[x['d_time']] for x in data if x['user'] == 'user{0}'.format(user_number)])
    T = np.array([[float(x[0] < 1), float(x[0] < 12.0), float(x[0] < 144.0)] for x in T])
    
    if X.shape[0] < 100:
        continue
    
    X_train, X_test = np.vsplit(X, [int(X.shape[0] * 0.8)])
    T_train, T_test = np.vsplit(T, [int(T.shape[0] * 0.8)])

    N_train = X_train.shape[0]
    N_test = X_test.shape[0]
    N_input = X_train.shape[1]
    N_output = 3

    tf.reset_default_graph()
    alpha = 0.0001
    n_epoch = int(X.shape[0] / 10)
    batchsize = 16
    x = tf.placeholder(tf.float32, [None, N_input])
    t = tf.placeholder(tf.float32, [None, N_output])

    y = MLP(x)
    cost = tf.reduce_mean(tf.square(y - t))
    optimizer = tf.train.AdamOptimizer(alpha).minimize(cost)

    # test用
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.greater(y, [0.5, 0.5, 0.5]), tf.greater(t, [0.5, 0.5, 0.5])), tf.float32))

    train_summary_loss = tf.summary.scalar('train_loss', cost)
    train_summary_accuracy = tf.summary.scalar('train_accuracy', accuracy)
    test_summary_loss = tf.summary.scalar('test_loss', cost)
    test_summary_accuracy = tf.summary.scalar('test_accuracy', accuracy)

    # optimizer = tf.train.AdamOptimizer(alpha).minimize(cost)

    # accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.greater(y * t_std + t_mean, [0.5]), tf.greater(t * t_std + t_mean, [0.5])), tf.float32))

    # train_summary_loss = tf.summary.scalar('train_loss', cost)
    # train_summary_accuracy = tf.summary.scalar('train_accuracy', accuracy)
    # test_summary_loss = tf.summary.scalar('test_loss', cost)
    # test_summary_accuracy = tf.summary.scalar('test_accuracy', accuracy)

    model_path = './models/model_user{0}/'.format(user_number)
    if not os.path.exists(model_path):
        os.mkdir(model_path)
    
    logs_path = './logs/log_user{0}/'.format(user_number)
    if os.path.exists(logs_path):
        shutil.rmtree(logs_path)
    os.mkdir(logs_path)
    saver = tf.train.Saver()
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    sum_n += 1
    tem_ac = 0
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, './models/model0/')
        max_accuracy = 0
        for epoch in range(n_epoch):
            print('epoch %d | ' % epoch, end='')
            sum_loss = 0
            perm = np.random.permutation(N_train)
            for i in range(0, N_train, batchsize):
                X_batch = X_train[perm[i:i+batchsize]]
                T_batch = T_train[perm[i:i+batchsize]]

                _, loss, train_loss, train_accuracy = sess.run([optimizer, cost, train_summary_loss, train_summary_accuracy], feed_dict={x:X_batch, t:T_batch})
                sum_loss += np.mean(loss) * X_batch.shape[0]
                summary_writer.add_summary(train_loss, epoch * N_train + len(perm[:i+batchsize]))
                summary_writer.add_summary(train_accuracy, epoch * N_train + len(perm[:i+batchsize]))

            loss = sum_loss / N_train
            accuracy_, test_loss, test_accuracy = sess.run([accuracy, test_summary_loss, test_summary_accuracy], feed_dict={x:X_test, t:T_test})
            summary_writer.add_summary(test_loss, epoch)
            summary_writer.add_summary(test_accuracy, epoch)
            print('Train loss %.4f | Test accuracy: %.4f' %(loss, accuracy_), end='')
            if max_accuracy < accuracy_:
                max_accuracy = accuracy_
                tem_ac = accuracy_
                saver.save(sess, model_path)
                print(' #', end='')
            print('')
    sum_ac += tem_ac
    print('Average Accuracy: ', sum_ac / sum_n)

INFO:tensorflow:Restoring parameters from ./models/model0/
epoch 0 | Train loss 0.1248 | Test accuracy: 0.8141 #
epoch 1 | Train loss 0.1153 | Test accuracy: 0.8333 #
epoch 2 | Train loss 0.1139 | Test accuracy: 0.8526 #
epoch 3 | Train loss 0.1146 | Test accuracy: 0.8397
epoch 4 | Train loss 0.1137 | Test accuracy: 0.8590 #
epoch 5 | Train loss 0.1137 | Test accuracy: 0.8654 #
epoch 6 | Train loss 0.1118 | Test accuracy: 0.8718 #
epoch 7 | Train loss 0.1097 | Test accuracy: 0.8718
epoch 8 | Train loss 0.1095 | Test accuracy: 0.8846 #
epoch 9 | Train loss 0.1105 | Test accuracy: 0.8782
epoch 10 | Train loss 0.1108 | Test accuracy: 0.8718
epoch 11 | Train loss 0.1117 | Test accuracy: 0.8718
epoch 12 | Train loss 0.1104 | Test accuracy: 0.8654
epoch 13 | Train loss 0.1099 | Test accuracy: 0.8846
epoch 14 | Train loss 0.1090 | Test accuracy: 0.8846
epoch 15 | Train loss 0.1103 | Test accuracy: 0.8718
epoch 16 | Train loss 0.1104 | Test accuracy: 0.8846
epoch 17 | Train loss 0.1100 | Test 

epoch 16 | Train loss 0.1279 | Test accuracy: 0.7477
epoch 17 | Train loss 0.1229 | Test accuracy: 0.8288
Average Accuracy:  0.8478945940732956
INFO:tensorflow:Restoring parameters from ./models/model0/
epoch 0 | Train loss 0.1324 | Test accuracy: 0.8158 #
epoch 1 | Train loss 0.1325 | Test accuracy: 0.8170 #
epoch 2 | Train loss 0.1255 | Test accuracy: 0.8158
epoch 3 | Train loss 0.1360 | Test accuracy: 0.8133
epoch 4 | Train loss 0.1346 | Test accuracy: 0.8158
epoch 5 | Train loss 0.1308 | Test accuracy: 0.7708
epoch 6 | Train loss 0.1353 | Test accuracy: 0.7887
epoch 7 | Train loss 0.1326 | Test accuracy: 0.7874
epoch 8 | Train loss 0.1310 | Test accuracy: 0.8170
epoch 9 | Train loss 0.1291 | Test accuracy: 0.8152
epoch 10 | Train loss 0.1315 | Test accuracy: 0.8145
epoch 11 | Train loss 0.1302 | Test accuracy: 0.7683
epoch 12 | Train loss 0.1246 | Test accuracy: 0.8176 #
epoch 13 | Train loss 0.1268 | Test accuracy: 0.7856
epoch 14 | Train loss 0.1270 | Test accuracy: 0.8170
epoch 

epoch 151 | Train loss 0.1272 | Test accuracy: 0.7862
epoch 152 | Train loss 0.1297 | Test accuracy: 0.8152
epoch 153 | Train loss 0.1308 | Test accuracy: 0.7733
epoch 154 | Train loss 0.1354 | Test accuracy: 0.7930
epoch 155 | Train loss 0.1280 | Test accuracy: 0.8028
epoch 156 | Train loss 0.1247 | Test accuracy: 0.8176
epoch 157 | Train loss 0.1268 | Test accuracy: 0.8170
epoch 158 | Train loss 0.1260 | Test accuracy: 0.8195
epoch 159 | Train loss 0.1274 | Test accuracy: 0.8201
epoch 160 | Train loss 0.1227 | Test accuracy: 0.8207 #
epoch 161 | Train loss 0.1311 | Test accuracy: 0.8176
epoch 162 | Train loss 0.1287 | Test accuracy: 0.7967
epoch 163 | Train loss 0.1312 | Test accuracy: 0.8195
epoch 164 | Train loss 0.1262 | Test accuracy: 0.8201
epoch 165 | Train loss 0.1289 | Test accuracy: 0.8059
epoch 166 | Train loss 0.1335 | Test accuracy: 0.8170
epoch 167 | Train loss 0.1305 | Test accuracy: 0.7924
epoch 168 | Train loss 0.1396 | Test accuracy: 0.8041
epoch 169 | Train loss 0.1

epoch 33 | Train loss 0.1633 | Test accuracy: 0.7628
epoch 34 | Train loss 0.1687 | Test accuracy: 0.7740
epoch 35 | Train loss 0.1687 | Test accuracy: 0.7596
epoch 36 | Train loss 0.1637 | Test accuracy: 0.7660
epoch 37 | Train loss 0.1671 | Test accuracy: 0.7821
epoch 38 | Train loss 0.1638 | Test accuracy: 0.7708
epoch 39 | Train loss 0.1627 | Test accuracy: 0.7516
epoch 40 | Train loss 0.1655 | Test accuracy: 0.7821
epoch 41 | Train loss 0.1711 | Test accuracy: 0.7628
epoch 42 | Train loss 0.1671 | Test accuracy: 0.7692
epoch 43 | Train loss 0.1627 | Test accuracy: 0.7676
epoch 44 | Train loss 0.1661 | Test accuracy: 0.7612
epoch 45 | Train loss 0.1651 | Test accuracy: 0.7837
epoch 46 | Train loss 0.1674 | Test accuracy: 0.7628
epoch 47 | Train loss 0.1655 | Test accuracy: 0.7644
epoch 48 | Train loss 0.1637 | Test accuracy: 0.7821
epoch 49 | Train loss 0.1696 | Test accuracy: 0.7147
epoch 50 | Train loss 0.1757 | Test accuracy: 0.7837
epoch 51 | Train loss 0.1683 | Test accuracy: 

epoch 82 | Train loss 0.1456 | Test accuracy: 0.7705
epoch 83 | Train loss 0.1681 | Test accuracy: 0.7571
epoch 84 | Train loss 0.1504 | Test accuracy: 0.7181
epoch 85 | Train loss 0.1499 | Test accuracy: 0.7781
epoch 86 | Train loss 0.1565 | Test accuracy: 0.7819
epoch 87 | Train loss 0.1519 | Test accuracy: 0.7790
epoch 88 | Train loss 0.1536 | Test accuracy: 0.7533
epoch 89 | Train loss 0.1638 | Test accuracy: 0.7686
epoch 90 | Train loss 0.1623 | Test accuracy: 0.7790
epoch 91 | Train loss 0.1560 | Test accuracy: 0.7819
epoch 92 | Train loss 0.1580 | Test accuracy: 0.7619
epoch 93 | Train loss 0.1508 | Test accuracy: 0.7667
epoch 94 | Train loss 0.1549 | Test accuracy: 0.7705
epoch 95 | Train loss 0.1491 | Test accuracy: 0.7810
epoch 96 | Train loss 0.1523 | Test accuracy: 0.6924
epoch 97 | Train loss 0.1554 | Test accuracy: 0.7552
epoch 98 | Train loss 0.1585 | Test accuracy: 0.7543
epoch 99 | Train loss 0.1493 | Test accuracy: 0.7695
epoch 100 | Train loss 0.1588 | Test accuracy:

...

INFO:tensorflow:Restoring parameters from ./models/model0/
epoch 0 | Train loss 0.2032 | Test accuracy: 0.7131 #
epoch 1 | Train loss 0.1877 | Test accuracy: 0.7342 #
epoch 2 | Train loss 0.1683 | Test accuracy: 0.7426 #
epoch 3 | Train loss 0.1657 | Test accuracy: 0.7426
epoch 4 | Train loss 0.1655 | Test accuracy: 0.7468 #
epoch 5 | Train loss 0.1655 | Test accuracy: 0.7426
epoch 6 | Train loss 0.1639 | Test accuracy: 0.7468
epoch 7 | Train loss 0.1616 | Test accuracy: 0.7468
epoch 8 | Train loss 0.1604 | Test accuracy: 0.7511 #
epoch 9 | Train loss 0.1577 | Test accuracy: 0.7468
epoch 10 | Train loss 0.1583 | Test accuracy: 0.7468
epoch 11 | Train loss 0.1572 | Test accuracy: 0.7511
epoch 12 | Train loss 0.1562 | Test accuracy: 0.7511
epoch 13 | Train loss 0.1562 | Test accuracy: 0.7511
epoch 14 | Train loss 0.1557 | Test accuracy: 0.7468
epoch 15 | Train loss 0.1540 | Test accuracy: 0.7511
epoch 16 | Train loss 0.1533 | Test accuracy: 0.7511
epoch 17 | Train loss 0.1529 | Test accu

epoch 11 | Train loss 0.1524 | Test accuracy: 0.7602
epoch 12 | Train loss 0.1520 | Test accuracy: 0.7602
epoch 13 | Train loss 0.1522 | Test accuracy: 0.7602
epoch 14 | Train loss 0.1519 | Test accuracy: 0.7602
epoch 15 | Train loss 0.1518 | Test accuracy: 0.7602
epoch 16 | Train loss 0.1517 | Test accuracy: 0.7602
epoch 17 | Train loss 0.1522 | Test accuracy: 0.7602
epoch 18 | Train loss 0.1528 | Test accuracy: 0.7602
epoch 19 | Train loss 0.1518 | Test accuracy: 0.7661
epoch 20 | Train loss 0.1525 | Test accuracy: 0.7544
epoch 21 | Train loss 0.1517 | Test accuracy: 0.7544
epoch 22 | Train loss 0.1520 | Test accuracy: 0.7602
epoch 23 | Train loss 0.1517 | Test accuracy: 0.7602
epoch 24 | Train loss 0.1515 | Test accuracy: 0.7602
epoch 25 | Train loss 0.1515 | Test accuracy: 0.7602
epoch 26 | Train loss 0.1516 | Test accuracy: 0.7544
epoch 27 | Train loss 0.1516 | Test accuracy: 0.7544
Average Accuracy:  0.8333157621897184
INFO:tensorflow:Restoring parameters from ./models/model0/
ep

INFO:tensorflow:Restoring parameters from ./models/model0/
epoch 0 | Train loss 0.1528 | Test accuracy: 0.8068 #
epoch 1 | Train loss 0.1500 | Test accuracy: 0.8068
epoch 2 | Train loss 0.1494 | Test accuracy: 0.8068
epoch 3 | Train loss 0.1493 | Test accuracy: 0.8068
epoch 4 | Train loss 0.1486 | Test accuracy: 0.8068
epoch 5 | Train loss 0.1485 | Test accuracy: 0.8019
epoch 6 | Train loss 0.1490 | Test accuracy: 0.8019
epoch 7 | Train loss 0.1487 | Test accuracy: 0.8019
epoch 8 | Train loss 0.1477 | Test accuracy: 0.8019
epoch 9 | Train loss 0.1479 | Test accuracy: 0.8019
epoch 10 | Train loss 0.1480 | Test accuracy: 0.8019
epoch 11 | Train loss 0.1470 | Test accuracy: 0.8019
epoch 12 | Train loss 0.1471 | Test accuracy: 0.8019
epoch 13 | Train loss 0.1473 | Test accuracy: 0.8019
epoch 14 | Train loss 0.1469 | Test accuracy: 0.8019
epoch 15 | Train loss 0.1467 | Test accuracy: 0.8019
epoch 16 | Train loss 0.1467 | Test accuracy: 0.8019
epoch 17 | Train loss 0.1465 | Test accuracy: 0.

INFO:tensorflow:Restoring parameters from ./models/model0/
epoch 0 | Train loss 0.1580 | Test accuracy: 0.7831 #
epoch 1 | Train loss 0.1589 | Test accuracy: 0.7831
epoch 2 | Train loss 0.1571 | Test accuracy: 0.7884 #
epoch 3 | Train loss 0.1543 | Test accuracy: 0.7884
epoch 4 | Train loss 0.1547 | Test accuracy: 0.7884
epoch 5 | Train loss 0.1537 | Test accuracy: 0.7884
epoch 6 | Train loss 0.1530 | Test accuracy: 0.7884
epoch 7 | Train loss 0.1532 | Test accuracy: 0.7884
epoch 8 | Train loss 0.1520 | Test accuracy: 0.7884
epoch 9 | Train loss 0.1529 | Test accuracy: 0.7884
epoch 10 | Train loss 0.1518 | Test accuracy: 0.7884
epoch 11 | Train loss 0.1517 | Test accuracy: 0.7884
epoch 12 | Train loss 0.1521 | Test accuracy: 0.7884
epoch 13 | Train loss 0.1519 | Test accuracy: 0.7884
epoch 14 | Train loss 0.1515 | Test accuracy: 0.7937 #
epoch 15 | Train loss 0.1512 | Test accuracy: 0.7937
epoch 16 | Train loss 0.1515 | Test accuracy: 0.7937
epoch 17 | Train loss 0.1515 | Test accuracy

epoch 3 | Train loss 0.1383 | Test accuracy: 0.7500 #
epoch 4 | Train loss 0.1377 | Test accuracy: 0.7500
epoch 5 | Train loss 0.1372 | Test accuracy: 0.7500
epoch 6 | Train loss 0.1368 | Test accuracy: 0.7500
epoch 7 | Train loss 0.1366 | Test accuracy: 0.7500
epoch 8 | Train loss 0.1368 | Test accuracy: 0.7500
epoch 9 | Train loss 0.1362 | Test accuracy: 0.7500
epoch 10 | Train loss 0.1363 | Test accuracy: 0.7500
Average Accuracy:  0.829675629734993
INFO:tensorflow:Restoring parameters from ./models/model0/
epoch 0 | Train loss 0.1583 | Test accuracy: 0.8000 #
epoch 1 | Train loss 0.1427 | Test accuracy: 0.8778 #
epoch 2 | Train loss 0.1347 | Test accuracy: 0.8444
epoch 3 | Train loss 0.1340 | Test accuracy: 0.8444
epoch 4 | Train loss 0.1340 | Test accuracy: 0.8444
epoch 5 | Train loss 0.1331 | Test accuracy: 0.8667
epoch 6 | Train loss 0.1322 | Test accuracy: 0.8667
epoch 7 | Train loss 0.1321 | Test accuracy: 0.8667
epoch 8 | Train loss 0.1310 | Test accuracy: 0.8444
epoch 9 | Tra

\`tensorboard --logdir ./tf/log/`

"localhost:6006"。